# Exploration of Road Elevation Estimation Methods

### Objective
Compare methods for estimating elevation at positions along roadways
### Source Data
USGS 3DEP 1/3 arc-second (~10M)

In [72]:
# import necessary packages/modules/libraries
import USGS3DEP
import USGSlookup
import layers

import numpy as np
from scipy import interpolate
import pandas as pd

from mpl_toolkits import mplot3d
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn
from matplotlib import cm

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

### Test Point:  *39.589093, -105.644230*
![Mt. Evans](img/MtEvans.png)

This location, the NW face of Mt. Evans (Colorado, USA,) was choosen for its steep terrain. The large elevation differentials will highlight deviations in our results using various methods. Although our final objective is to apply this technique to roadways, this location is well suited for comparing the accuracy of various elevation estimation methods.

<br>
<br>
<br>
## Comparison of Query Methods

### Query Method 1: USGS Point Query API
The USGS Elevation Point Query Service allows us the flexibility to query an point in contiguous United States and returns an elevation value. The source data is 1/3 arc-second (~10M) resolution. The following excerpt, from a USGS FAQ page, suggests that the API is using an interpolation method to estimate elevations not habitating a data point.

***Q:***  *How accurate are elevations generated by the elevation point query service?*

***A:***  *The elevation point query service returns elevations from the 1/3 arc-second 3DEP DEM dataset in The National Map. The 1/3 arc-second data has 10m x 10m cell grid resolution, and may contain resampled elevation data using bilinear interpolation from higher resolution sources (if they exist). Spot elevations are not official and do not represent precisely measured ground surveyed values. Spot elevations derived from The National Map 3DEP DEMs at a specific location might differ from monumented control published on USGS topographic maps or cited in National Geodetic Survey Marks and Datasheets. Variances between spot elevations and surveyed elevations may be evident for features, such as mountain peaks or summits, and where the local relief (rate of change of elevation) is more prominent. For most purposes other than engineering, interpolated spot elevation values are sufficiently accurate.*

[USGS FAQ page on their Website]

#### Elevation value returned from Point Query API (in feet):

In [73]:
testPnt = (39.589093, -105.644230)
testLat = testPnt[0]
testLon = testPnt[1]

USGS3DEP.ElevQuery10M(testLat, testLon)

13687.63

### Query Method 2: Point Query from NREL's Raster Database

The raster database contains 1/3 arc-second (~10M) pixels, stored in 1deg x 1deg tiles. We would expect to see a "snap-to-nearest" type behavior with no filtering, smoothing or interpolating occuring. Most likely this format will be superior due to the end-to-end control the user will retain over the data. 

#### Elevation value returned from Raster Database

In [74]:
gridRef = USGSlookup.build_grid_refs([testLat],[testLon])[0]
USGSlookup.get_elev_data(gridRef, [testLat], [testLon])

Retreiving Path to Raster Data...


Exception: Incorrect FilePath to Raster Database, or data does not exist.

### Results:
Our initial findings suggest that both datasets will return the same values

#### Lets continue to investigate...
<br>
<br>
<br>
<br>

## A 1 Meter Walk in all Cardinal Directions
Our next experiment will test 4 points located approximately 1 meter, in each direction, from the original point. This will hopefully provide more insight into the methods used by each dataset.

### Query Method 1: USGS Point Query API


In [ ]:
approxOneMeter = .00000925925925 # approximately 1 meter in degrees of lattitude

print('1M North:')
print(USGS3DEP.ElevQuery10M(testLat+approxOneMeter, testLon))

print('1M South:')
print(USGS3DEP.ElevQuery10M(testLat-approxOneMeter, testLon))

print('1M East:')
print(USGS3DEP.ElevQuery10M(testLat, testLon+approxOneMeter))

print('1M West:')
print(USGS3DEP.ElevQuery10M(testLat, testLon-approxOneMeter))

### Query Method 2: Point Query from NREL's Raster Database


In [ ]:
print('1M North:')
print(USGSlookup.get_elev_data(gridRef, [testLat+approxOneMeter], [testLon])[0])

print('1M South:')
print(USGSlookup.get_elev_data(gridRef, [testLat-approxOneMeter], [testLon])[0])

print('1M East:')
print(USGSlookup.get_elev_data(gridRef, [testLat], [testLon+approxOneMeter])[0])

print('1M West:')
print(USGSlookup.get_elev_data(gridRef, [testLat], [testLon-approxOneMeter])[0])

### Results:
Due to extreme steepness of the terrain at our choosen point, we expect to see variations in elevation values when we offset our query by a meter. However, with either query method, we do not. This suggests that maybe neither is interpolating to produce it's results. 

It's possible instead, that they are returning the datapoint value for the entire 10M x 10M tile for which that value is associated. In other words, a "snap-to-nearest" method is possibly being utilized.

#### Investigating further...
<br>
<br>
<br>


## A Test with a 20m x 20m Land Area

### Query Method 1: USGS Point Query API


In [ ]:
def query_20M_tile(start_lat, start_lon):
    tile_20M = []
    lat = start_lat
    for row in range(20):
        lon = start_lon
        row_list = [np.nan] * 20
        for col in range(20):
            row_list[col] = USGS3DEP.ElevQuery10M(lat, lon)
            lon += approxOneMeter
        tile_20M += [row_list]
        lat -= approxOneMeter
    return tile_20M
        
twenty_by_twenty_API = query_20M_tile(testLat, testLon)

#### Visualizing the Values

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.gca(projection='3d')
ax.set_title('20m x 20m Area on Mt. Evans - from query API (in feet)')

x = np.arange(0, 20 * 3.280839895, 3.280839895) # spans 20m in feet
y = np.arange(0, 20 * 3.280839895, 3.280839895) # spans 20m in feet
x, y = np.meshgrid(x, y)

z_API = np.array(twenty_by_twenty_API)

surf = ax.plot_surface(x, y, z_API, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=1, antialiased=False)

### Query Method 2: Point Query from NREL's Raster Database


In [ ]:
def raster_20M_tile(start_lat, start_lon):
    tile_20M = []
    lat = start_lat
    for row in range(20):
        lon = start_lon
        row_list = [np.nan] * 20
        for col in range(20):
            row_list[col] = USGSlookup.get_elev_data(gridRef, [lat], [lon])[0];
            lon += approxOneMeter
        tile_20M += [row_list]
        lat -= approxOneMeter
    return tile_20M
        
twenty_by_twenty_raster = raster_20M_tile(testLat, testLon)

#### Visualizing the Results

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.gca(projection='3d')
ax.set_title('20m x 20m Area on Mt. Evans - from Raster Database (in feet)')

z_raster = np.array(twenty_by_twenty_raster)

surf = ax.plot_surface(x, y, z_raster, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

### The Difference Between the Results

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.gca(projection='3d')
ax.set_title('20m x 20m Area on Mt. Evans - The Difference Between the Two (in feet)')
ax.set_zlim(0,80)

z_diff = np.subtract(z_API, z_raster)

surf = ax.plot_surface(x, y, z_diff, rstride=1, cstride=1, cmap=cm.coolwarm,
                       linewidth=0, antialiased=True)

### Results:


## Estimation Method 1: Bilinear Interpolation using the four adjacent data points
![Bilinear Interpolation](img/2Dinterp.png)

### Data Sources for Method Analysis

### 1. USGS Point Query API

In [ ]:
# simulated vehicle trace, with lat/lon fixes, traveling up Lookout Mt Rd, Golden CO
trace = [(39.742040, -105.232647), (39.742927, -105.233657), (39.742844, -105.23484), (39.743760, -105.234487), (39.744537, -105.234848), (39.744620, -105.236328), (39.745758, -105.236003), (39.746673, -105.236905), (39.747284, -105.237049), (39.747672, -105.238204), (39.749087, -105.238565), (39.749309, -105.240442), (39.748588, -105.242571), (39.748865, -105.244700), (39.748144, -105.242354), (39.747950, -105.244231), (39.747534, -105.241957), (39.748061, -105.240189),(39.745352, -105.239062), (39.742943, -105.240006), (39.742316, -105.239233), (39.741177, -105.239855), (39.740286, -105.242302), (39.739560, -105.242988), (39.738851, -105.242795), (39.736964, -105.245316), (39.735674, -105.244360), (39.733276, -105.244459), (39.733247, -105.244815), (39.733895, -105.244800), (39.733942, -105.245020), (39.733236, -105.245126), (39.733306, -105.245369), (39.734391, -105.245339), (39.735365, -105.245513), (39.735324, -105.245801), (39.734367, -105.245794), (39.733667, -105.246241), (39.732611, -105.245521), (39.732150, -105.244883), (39.731964, -105.243746), (39.731512, -105.243279), (39.732092, -105.242570), (39.732684, -105.242209), (39.732748, -105.241519), (39.732903, -105.241007)]

# create a list of elevation values from the USGS query API
elev_trace_query = []
for i in range(len(trace)):
    elev = USGS3DEP.ElevQuery10M(trace[i][0], trace[i][1])
    elev_trace_query += [elev]
    
print(elev_trace_query)


### 2. USGS Raster Database

In [ ]:
reload(USGSlookup)

gridRef = USGSlookup.build_grid_refs([trace[0][0]],[trace[0][1]])[0]

# create a list of elevation values from the raster database
elev_trace_raster = []
for i in range(len(trace)):
    elev = USGSlookup.get_elev_data(gridRef, [trace[i][0]], [trace[i][1]])[0];
    elev_trace_raster += [elev]
    
print(elev_trace_raster)    


### 3. USGS Raster Database Bilinearly Interpolated

In [ ]:
# create a Tile object for the 1deg x 1deg tile containing our trace
n40w106 = layers.Tile([39.742040], [-105.232647])

# bilinearly interpolate elevation information, then append it to the trace data
trace_interp = n40w106.elev_walk(trace);

# isolate elevation values in seperate string
elev_trace_interp = []
for i in range(len(trace_interp)):
    elev = trace_interp[i][2]
    elev_trace_interp += [elev]

print(elev_trace_interp)    

### Elevation Profile Comparison in 2D

In [ ]:
fig=plt.figure(figsize=(20,20))
fig.show()

x_vals = np.linspace(0, 100, num=len(trace), endpoint=False)

ax = plt.axes()
ax.set_title('2D Elevation Profile - Lookout Mt Rd')

ax.plot(x_vals, elev_trace_query, c='y', label='API Query')
ax.plot(x_vals, elev_trace_raster, c='g', label='Raster DB')
ax.plot(x_vals, elev_trace_interp, c='b', label='Bilinear Interpolation')

plt.legend(loc=2)

### Elevation Profile Comparison in 3D

In [ ]:
lon_data = np.empty(len(trace_interp))
lat_data = np.empty(len(trace_interp))

for i in range(len(trace_interp)):
    lon_data[i] = trace_interp[i][1] # lon array
    lat_data[i] = trace_interp[i][0] # lat array

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = plt.axes(projection='3d')
ax.set_title('3D Elevation Profile - Lookout Mt Rd')

ax.plot3D(lon_data, lat_data, elev_trace_query, c='y', label='API Query')
ax.plot3D(lon_data, lat_data, elev_trace_raster, c='g', label='Raster DB')
ax.plot3D(lon_data, lat_data, elev_trace_interp, c='b', label='Bilinear Interpolation')

plt.legend(loc=2)

In [ ]:
from math import sqrt, sin, asin, cos, atan2, radians
 
def append_elev(trace, elev_list):
    appended_trace = []
    for i in range(len(trace)):
        lat = trace[i][0]
        lon = trace[i][1]
        elev = elev_list[i]
        appended_trace += [[lat, lon, elev]]
    return appended_trace    
             
def haversine(origin, destination):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # extract lats/lons from trace
    lat1, lon1, elev1 = origin
    lat2, lon2, elev2 = destination
    
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers.
    dist_km =  c * r
    dist_m = dist_km * 1000
    return dist_m
    
def slope_between_points(trace_with_elev, iterator):
    origin = trace_with_elev[iterator]
    destination = trace_with_elev[iterator + 1]
    rise_ft = destination[2] - origin[2]
    rise = rise_ft / 3.280839895 # convert ft to meters
    run = haversine(origin, destination)
    return ((rise / run) * 100)

def get_slope_list(trace_with_elev, append_to_trace=False, **kwargs):
    
    # return a list containing only slope
    if append_to_trace == False:
        slope_list = []
        for i in range(len(trace_with_elev) - 1):
            slope = slope_between_points(trace_with_elev, i)
            slope_list += [slope]
        return slope_list
    
    # return a list containing lat, lon, elev, slope
    else:
        trace_with_grade = []
        for i in range(len(trace_with_elev) - 1):
            slope = slope_between_points(trace_with_elev, i)
            trace_with_grade[i] = [trace_with_elev[i]] + [slope]
        return     
        

trace_query_elev = append_elev(trace, elev_trace_query)
trace_query_grade = get_slope_list(trace_query_elev, append_to_trace=False)

trace_raster_elev = append_elev(trace, elev_trace_raster)
trace_raster_grade = get_slope_list(trace_raster_elev, append_to_trace=False)

trace_interp_elev = append_elev(trace, elev_trace_interp)
trace_interp_grade = get_slope_list(trace_interp_elev, append_to_trace=False)

### Grade Profile Comparison

In [ ]:
fig=plt.figure(figsize=(20,20))
fig.show()

x_vals = np.linspace(0, 100, num=len(trace) - 1, endpoint=False)

ax = plt.axes()
ax.set_title('Grade Profile - Lookout Mt Rd')

ax.plot(x_vals, trace_query_grade, c='y', label='API Query')
ax.plot(x_vals, trace_raster_grade, c='g', label='Raster DB')
ax.plot(x_vals, trace_interp_grade, c='b', label='Bilinear Interpolation')

plt.legend(loc=2)

In [ ]:
def get_grade_sign(grade_trace):
    grade_sign = []
    for i in range(len(grade_trace)):
        if grade_trace[i] > 0:
            grade_sign += [1]
        elif grade_trace[i] < 0:
            grade_sign += [-1]
        else:
            grade_sign += [0]
    return grade_sign

query_concav = get_grade_sign(trace_query_grade)
raster_concav = get_grade_sign(trace_raster_grade)
interp_concav = get_grade_sign(trace_interp_grade)

In [ ]:
plt.scatter(x_vals, query_concav, c="y", alpha=1.0, marker=r'o')
plt.xlabel("Sign")
plt.ylabel("Leg")
plt.legend(loc=2)
plt.title('Query API Concavity')
plt.show()

plt.scatter(x_vals, raster_concav, c="g", alpha=1.0, marker=r'o')
plt.xlabel("Sign")
plt.ylabel("Leg")
plt.legend(loc=2)
plt.title('Raster DB Concavity')
plt.show()

plt.scatter(x_vals, interp_concav, c="b", alpha=1.0, marker=r'o')
plt.xlabel("Sign")
plt.ylabel("Leg")
plt.legend(loc=2)
plt.title('Interpolated Concavity')
plt.show()

In [ ]:
mt_evans = n40w106.slice_subregion((39.602595, -105.693718), (39.558016, -105.610281))

In [ ]:
rows = mt_evans.shape[0]
cols = mt_evans.shape[1]

X = np.arange(0, 10*cols, 10)
Y = np.arange(0, 10*rows, 10)
X, Y = np.meshgrid(X, Y)

Z = mt_evans.values.tolist()

data = [go.Surface(x=X, y=Y, z=Z, colorscale='Viridis')]

layout = go.Layout(
    width=800,
    height=700,
    autosize=False,
    title='Mt Evans Area - not scaled',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename='pandas-3d-surface', validate=False)


In [ ]:
def scale_blockworld(elevDF):
    elev_array = elevDF.values
    elev_array = np.repeat(elev_array, 10, axis=0)
    elev_array = np.repeat(elev_array, 10, axis=1)
    return pd.DataFrame(elev_array)

block_mt_evans = scale_blockworld(mt_evans)

Z_block = block_mt_evans.values.tolist()

data = [go.Surface(x=X, y=Y, z=Z_block, colorscale='Viridis')]

layout = go.Layout(
    width=800,
    height=700,
    autosize=False,
    title='Mt Evans Area - scaled - Block World',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        aspectratio = dict( x=1, y=1, z=0.7 ),
        aspectmode = 'manual'
    )
)

fig = dict(data=data, layout=layout)

iplot(fig, filename='pandas-3d-surface', validate=False)